**Entity Embeddings of Categorical Variables**
> https://arxiv.org/abs/1604.06737

**Generating Live Soccer-Math commentary from Play Data**
> https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwicl4Xi5J7wAhV7GDQIHW5lAOoQFjACegQIAxAD&url=https%3A%2F%2Fojs.aaai.org%2Findex.php%2FAAAI%2Farticle%2Fview%2F4691%2F4569&usg=AOvVaw0f2DIG3tjdsCeu_CJQ_2qW


In [5]:
!pip install fastcore
!pip install nbdev

     |████████████████████████████████| 46 kB 413 kB/s eta 0:00:011
     |████████████████████████████████| 455 kB 5.9 MB/s eta 0:00:01
     |████████████████████████████████| 49 kB 609 kB/s  eta 0:00:01
  Attempting uninstall: nbconvert
    Found existing installation: nbconvert 6.0.7
    Uninstalling nbconvert-6.0.7:
      Successfully uninstalled nbconvert-6.0.7


In [6]:
import pandas as pd
import numpy as np
import torch
from fastai.tabular.all import *
from fastcore.utils import *

# Load Data

In [2]:
df = pd.read_pickle('event_data.pickle')
df.head()

,time,event,player_sub,main_player,commentary
0,89',Yellow Card,,K. Phillips,A hasty challenge from Kalvin Phillips now and...
1,87',Substitution,H. Kane,Carlos Vinícius,Harry Kane - who became a father this week - m...
2,85',Yellow Card,,P. Højbjerg,"Hojbjerg picks up a late booking here, as the ..."
3,78',Substitution,T. Ndombèlé,Lucas Moura,"Tottenham make their second change now, with N..."
4,76',Substitution,H. Winks,M. Sissoko,Lloris gets a glance to this one and Ayling's ...


# Preprocess data

In [20]:
def process_time(time):
    sp = time.strip().split("'")
    minutes = int(sp[0])
    if len(sp[1]) > 0:
        time_to_add = int(sp[1].strip().split('+')[1].strip())
        minutes += time_to_add
    return minutes

df['pr_time'] = df['time'].apply(lambda x : process_time(x))

In [21]:
df.head()

,time,event,player_sub,main_player,commentary,pr_time
0,89',Yellow Card,,K. Phillips,"A hasty challenge from Kalvin Phillips now and his booking is more costly than usual; it is his fifth, earning him a one-match ban.",89
1,87',Substitution,H. Kane,Carlos Vinícius,"Harry Kane - who became a father this week - makes way for Tottenham's final change, with Carlos Vinicius on his place.",87
2,85',Yellow Card,,P. Højbjerg,"Hojbjerg picks up a late booking here, as the minutes tick down. Tottenham will be pleased with this though; a winning start to 2021 is just what they need.",85
3,78',Substitution,T. Ndombèlé,Lucas Moura,"Tottenham make their second change now, with Ndombele making way for Lucas Moura.",78
4,76',Substitution,H. Winks,M. Sissoko,Lloris gets a glance to this one and Ayling's return ball sails wide of the mark. Moussa Sissoko replaces Winks for the hosts.,76


In [30]:
# df = df.drop('pr_player_sub', axis=1)
df['player_sub'] = df['player_sub'].replace('', 'na')

In [31]:
df.head()

,time,event,player_sub,main_player,commentary,pr_time
0,89',Yellow Card,na,K. Phillips,"A hasty challenge from Kalvin Phillips now and his booking is more costly than usual; it is his fifth, earning him a one-match ban.",89
1,87',Substitution,H. Kane,Carlos Vinícius,"Harry Kane - who became a father this week - makes way for Tottenham's final change, with Carlos Vinicius on his place.",87
2,85',Yellow Card,na,P. Højbjerg,"Hojbjerg picks up a late booking here, as the minutes tick down. Tottenham will be pleased with this though; a winning start to 2021 is just what they need.",85
3,78',Substitution,T. Ndombèlé,Lucas Moura,"Tottenham make their second change now, with Ndombele making way for Lucas Moura.",78
4,76',Substitution,H. Winks,M. Sissoko,Lloris gets a glance to this one and Ayling's return ball sails wide of the mark. Moussa Sissoko replaces Winks for the hosts.,76


# Entity Embedding for Categorical Variable

In [33]:
cont, cat = cont_cat_split(df, dep_var='commentary')
cont, cat

(['pr_time'], ['time', 'event', 'player_sub', 'main_player'])

In [51]:
procs_nn = [Categorify]
device = torch.device('cpu')
splits = RandomSplitter(seed=23)(df)
to_nn = TabularPandas(df, procs_nn, cat,
                      splits=splits, y_names='commentary')
dls = to_nn.dataloaders(800, device=device)

In [53]:
learn = tabular_learner(dls, n_out=len(df['commentary']))
learn.fit_one_cycle(8, 5e-4)

epoch,train_loss,valid_loss,time
0,10.246932,10.516588,00:10
1,8.743799,8.352613,00:10
2,5.243084,0.589417,00:10
3,2.355135,0.105655,00:10
4,1.089481,0.086224,00:10
5,0.526291,0.079526,00:11
6,0.270160,0.077176,00:11
7,0.154150,0.076536,00:11


In [58]:
learn.save('learn8')

Path('models/learn8.pth')

In [59]:
def embed_features(learner, x):
    x = x.copy()
    for i, col in enumerate(learn.dls.cat_names):
        embed = learn.model.embeds[i]
        embed_data = embed(tensor(x[col], dtype=torch.int64).to(device))
        embed_names = [f'{col}_{j}' for j in range(embed_data.shape[1])]
        features = pd.DataFrame(data=embed_data, index=x.index, columns=embed_names)
        x = x.drop(col, axis=1)
        x = x.join(features)
    return x

In [60]:
procs = [Categorify]
to = TabularPandas(df, procs, cat, cont, 'commentary', splits=splits)

In [62]:
learn = learn.load('learn8')
embed_xs = embed_features(learn, to.train.xs)

In [63]:
embed_xs

,pr_time,time_0,time_1,time_2,time_3,time_4,time_5,time_6,time_7,time_8,...,main_player_20,main_player_21,main_player_22,main_player_23,main_player_24,main_player_25,main_player_26,main_player_27,main_player_28,main_player_29
26396,17,-0.006968,-0.008737,-0.002296,-0.014454,0.018661,0.003139,-0.004546,-0.011193,-0.000824,...,-0.010556,-0.001683,0.003408,0.001707,-0.002516,-0.003062,0.011289,0.001556,-0.006472,-0.002546
38745,87,-0.008931,0.014219,0.004200,0.015716,0.007030,-0.014227,0.003506,0.012484,-0.014160,...,-0.004266,-0.002528,-0.006009,-0.013528,-0.003304,0.006879,-0.012292,-0.013334,-0.006098,-0.008574
36803,15,0.001548,0.005927,0.008035,-0.006785,-0.010640,-0.026798,-0.014577,-0.003889,-0.019220,...,-0.006466,-0.006537,-0.004697,-0.002124,0.006044,0.009152,0.009950,0.006844,-0.009323,-0.000331
5666,9,-0.015175,-0.018255,0.001379,0.010012,0.019067,-0.005179,0.001627,0.001815,-0.001467,...,-0.013155,0.012651,-0.006049,0.001370,0.005356,0.000860,0.019063,-0.005408,-0.011974,-0.013826
15343,38,-0.002520,-0.008065,0.002077,0.004683,-0.006135,0.014256,-0.004571,0.019500,-0.008193,...,0.004878,0.001890,-0.012160,-0.004752,0.007116,0.012678,-0.000809,0.004747,0.005972,-0.011358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8416,82,-0.010255,-0.003131,-0.002138,0.007612,0.011863,-0.001542,0.003282,0.009329,-0.004910,...,0.012813,0.001940,-0.007500,-0.015081,-0.008530,-0.021439,-0.002683,0.010737,0.011484,0.000884
6418,61,-0.019200,-0.002456,0.006642,0.025335,-0.013232,0.007828,0.002839,0.005641,0.021768,...,-0.010052,0.020335,-0.006076,-0.002502,-0.001323,0.000017,-0.003714,-0.004403,-0.003647,0.003428
18010,53,-0.015372,0.007919,-0.013604,-0.007334,-0.016128,0.007268,0.006772,0.002357,0.011394,...,-0.002619,0.010359,-0.006357,0.008357,-0.009039,-0.008515,0.001211,0.001390,0.010677,0.002190
34531,65,0.016233,0.003198,-0.007764,-0.004424,-0.021971,0.014320,0.022195,-0.005421,0.005467,...,0.001325,-0.006893,-0.009253,0.013860,-0.001445,-0.022257,0.013199,0.011240,0.000492,0.009503


In [64]:
embed_valid_xs = embed_features(learn, to.valid.xs)

In [65]:
embed_valid_xs

,pr_time,time_0,time_1,time_2,time_3,time_4,time_5,time_6,time_7,time_8,...,main_player_20,main_player_21,main_player_22,main_player_23,main_player_24,main_player_25,main_player_26,main_player_27,main_player_28,main_player_29
26614,37,-0.001571,-0.005194,0.020086,-0.010233,0.017314,0.012149,0.017869,-0.000413,-0.002334,...,0.018499,-0.009874,-0.014684,0.004497,0.020075,-0.018890,0.009378,-0.001703,0.014393,0.007037
28533,83,0.005207,0.021995,0.002193,0.008718,-0.011274,0.000906,-0.003526,0.004498,0.000990,...,0.027776,0.008046,-0.006130,0.004404,-0.018272,0.022238,-0.009440,-0.008602,0.001524,0.003431
11568,71,-0.002151,-0.012435,0.007695,0.009055,-0.005768,0.000053,-0.005699,-0.004532,-0.008748,...,-0.001215,0.012397,0.007559,0.008509,-0.003347,0.022145,0.022286,-0.009576,0.004040,-0.006320
7628,72,0.003889,0.018350,0.001300,-0.007680,-0.004493,0.002557,0.015886,-0.003740,-0.007262,...,0.004563,-0.018718,-0.017428,0.001817,-0.003944,0.013369,-0.009413,-0.005916,-0.005188,0.004280
22395,95,0.011870,0.011980,-0.012554,0.000491,-0.026834,-0.005475,0.012536,0.011681,0.002556,...,0.009273,-0.011865,-0.015612,-0.006133,0.002135,-0.002489,-0.000704,0.004675,-0.018174,0.014102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11142,67,0.007186,-0.001696,0.016993,0.009635,-0.006630,0.021616,-0.001416,0.014099,-0.007209,...,0.014055,-0.002286,0.003382,0.012861,-0.016800,-0.008712,-0.010851,-0.002805,0.002223,0.022836
30134,78,0.002382,-0.011573,-0.002028,-0.006220,-0.016192,-0.007780,-0.021638,-0.007255,0.007257,...,-0.005423,0.007108,-0.007153,0.003900,0.000101,-0.007415,-0.006821,0.002518,0.009431,0.005331
18378,44,0.014752,-0.006680,-0.010279,0.005542,-0.016025,0.002630,-0.016002,0.012767,0.001007,...,-0.006742,-0.018365,-0.012713,-0.008808,0.007912,0.007905,-0.000963,-0.004839,0.004024,0.008467
15645,52,0.005664,-0.002775,-0.010530,0.017809,-0.022606,-0.007575,0.014757,0.014880,0.017331,...,-0.000416,0.005884,-0.000292,-0.025493,-0.016496,-0.019868,0.016401,0.025543,0.004366,0.007546


In [68]:
save_pickle('data/emb_xs', embed_xs)

In [69]:
save_pickle('data/emb_valid_xs', embed_valid_xs)

# Multi-Layer Perceptron

In [194]:
class Model(torch.nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Model, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.fc1 = torch.nn.Linear(self.input_size,
                                   self.hidden_size)
        self.tanh = torch.nn.Tanh()
        self.dropout = torch.nn.Dropout(p=0.5)
        self.fc2 = torch.nn.Linear(self.hidden_size, 500)
        self.batch_norm = torch.nn.BatchNorm1d(500)
        self.tanh2 = torch.nn.Tanh()
        self.fc3 = torch.nn.Linear(500, 256, bias=False)
#         self.softmax = torch.nn.Softmax()
        
    def forward(self, x):
        fc1 = self.fc1(x)
        tanh = self.tanh(fc1)
        dropout = self.dropout(tanh)
        fc2 = self.fc2(dropout)
        batch = self.batch_norm(fc2)
        tanh = self.tanh2(batch)
        output = self.fc3(tanh)
#         output = self.softmax(output)
        return output

In [190]:
x_train = embed_xs.iloc[:25000]
x_test = embed_xs.iloc[25000:]
y_train = to.train.ys[:25000]
y_test = to.train.ys[25000:]

In [191]:
x_train = torch.FloatTensor(x_train.values)
x_test = torch.FloatTensor(x_test.values)
y_train = torch.FloatTensor(y_train.values)
y_test = torch.FloatTensor(y_test.values)

In [195]:
model = Model(x_train.shape[1], 500)
criterion = torch.nn.KLDivLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [91]:
x_train.shape

torch.Size([25000, 77])

In [196]:
model.eval()

Model(
  (fc1): Linear(in_features=77, out_features=500, bias=True)
  (tanh): Tanh()
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=500, out_features=500, bias=True)
  (batch_norm): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (tanh2): Tanh()
  (fc3): Linear(in_features=500, out_features=256, bias=False)
)

In [198]:
model.eval()
hidden = model(x_train)
hidden.shape

torch.Size([25000, 256])

In [202]:
hidden[0].shape[0]

256

In [200]:
model.fc3.weight.shape

torch.Size([256, 500])

In [ ]:
"""
    Applying Gate mechanism
"""

for i in 

In [206]:
t = torch.randn(4,4)
b = t.view(2,8)
t, b

(tensor([[-0.1886, -0.0316,  0.0710,  0.0364],
         [-1.6100, -0.2935,  0.0307,  0.0273],
         [ 0.3379,  0.0906, -1.3006,  0.0655],
         [ 0.4121,  0.4271, -0.7854, -0.9690]]),
 tensor([[-0.1886, -0.0316,  0.0710,  0.0364, -1.6100, -0.2935,  0.0307,  0.0273],
         [ 0.3379,  0.0906, -1.3006,  0.0655,  0.4121,  0.4271, -0.7854, -0.9690]]))

In [211]:
lstm = torch.nn.LSTM(hidden[0].shape[0], 1)

In [212]:
out, h = lstm(hidden)

RuntimeError: input must have 3 dimensions, got 2

# Tensorflow

In [184]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Activation
from sklearn.model_selection import train_test_split

x_tr, x_te, y_tr, y_te = train_test_split(df.drop('commentary', axis=1), df['commentary'],
                                          test_size=0.33)


encoder_in = Input(shape=(len(x_tr.columns)-1,))
embedding = Embedding(2000, 128) (encoder_in)
encoder = LSTM(64, return_sequences=False)(embedding)

model = Model(encoder_in, Activation('softmax'))
model.compile(loss=keras.losses.CategoricalCrossEntropy(),
              optimizer=keras.optimizer.Adam(1e-3),
              metrics=['accuracy'])

model.summary()
# def encoder

TypeError: embedding(): argument 'indices' (position 2) must be Tensor, not KerasTensor

In [179]:
x_te

,time,event,player_sub,main_player,pr_time
8278,42',Yellow Card,na,L. Ayling,42
18250,77',Substitution,J. Maddison,Ayoze Pérez,77
23807,44',Yellow Card,na,I. Diallo,44
1673,64',Substitution,Nouri C. Coady,R. Aït,64
5334,60',Substitution,Rúben Neves,M. Gibbs-White,60
...,...,...,...,...,...
14687,82',Substitution,J. Bowen,A. Yarmolenko,82
32707,77',Substitution,J. Maddison,Ayoze Pérez,77
36464,82',Substitution,J. Bowen,A. Yarmolenko,82
12392,90' + 1',Substitution,J. Vardy,K. Iheanacho,91
